In [1]:
# My Standard Spark Session!

# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row


spark = SparkSession.builder.appName("myapp").getOrCreate()

#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [2]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 Python Version:  3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
    Python Path:  C:\ProgramData\Anaconda3\envs\py36
 My Python Libs:  C:\Users\MerzDal/.myconfigs
   My Spark Dir:  C:\Users\MerzDal/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x000001EA60E1E9B0>


In [4]:
# 6 columns: strings, ints, float
import random

num = 75000


# Lists / arrays
lst_months = ['January','Feburary','March','April','May','June',
              'July','August','September','October','November','December']

indices = [int(x) for x in np.linspace(0,num-1,num)]
# print(indices)

years = [random.randint(1900,2019) for x in range(num)]
# print(years)

ages = [(2019 - x) for x in years]
# print(ages)

months = [random.choice(lst_months) for x in range(num)]
# print(months)

worth = [random.random() * 100000 for x in range(num)]
# print(worth)

lst_names = ['sarah','bill','steve','mary','alyssa','brian','elizabeth','josh','ryan','katie','connor',
             'erica','lisa','doug','stacie','chris','gary','tom','ellen','paula']
names = [random.choice(lst_names) for x in range(num)]



# Create Array!
df = DFB.arrays_to_dataframe([indices,names,months,years,ages,worth],
                             ['index','name','month','year','age','net_worth'])


df.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,0,katie,Feburary,1949,70,"33,919.04"
1,1,alyssa,October,2004,15,"41,329.12"
2,2,katie,Feburary,1974,45,"83,141.61"
3,3,elizabeth,December,2006,13,"48,100.06"


# Working Example:

## Let's categorize each sale as low, medium, or high
- low: <= 20,000
- medium: 20,000 < x < 34,000
- high: 34,000 <= 100,000

df.withColumn("

In [5]:
path_out = "C:/Users/MerzDal/Documents/testing/data/raw/"

In [6]:
print(path_out)

C:/Users/MerzDal/Documents/testing/data/raw/


In [9]:
if not os.path.exists(path_out):
    os.makedirs(path_out)
    os.listdir(path_out)

In [8]:
df.coalesce(1)\
.write\
.mode("overwrite")\
.parquet(path_out)

### Testing:

In [11]:
df = ctx.spark.read.parquet(path_out)

In [12]:
df.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,0,katie,Feburary,1949,70,"33,919.04"
1,1,alyssa,October,2004,15,"41,329.12"
2,2,katie,Feburary,1974,45,"83,141.61"
3,3,elizabeth,December,2006,13,"48,100.06"


In [13]:
df.count()

75000

In [14]:
df1 = df.filter(col("net_worth") > 60000)

In [15]:
df1.count()

30031

In [16]:
df1.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,2,katie,Feburary,1974,45,"83,141.61"
1,6,stacie,December,1904,115,"66,419.96"
2,7,gary,July,1983,36,"80,843.95"
3,8,paula,April,1951,68,"79,409.42"


In [30]:
df2 = df1\
.filter(col("net_worth") > 80000)\
.filter(col("name").startswith("e"))

In [31]:
df2.count()

2295

In [32]:
df2.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,89,elizabeth,January,2004,15,"85,136.63"
1,102,elizabeth,August,1923,96,"97,428.25"
2,104,erica,March,1942,77,"81,693.73"
3,113,ellen,July,1941,78,"87,176.48"
